# 3.モンテカルロ法

In [11]:
import gym
import numpy as np

env = gym.make('FrozenLake-v0', is_slippery=False)
env.render()


SFFF
FHFH
FFFH
HFFG


* 「ゲームをやってみた結果、もらった報酬」を元に価値計算
* 今回の「価値」は、Q(s, a)で表される、「行動の価値」

In [2]:
class MonteCarlo:
    def __init__(self, env):
        self.env = env
        self.Q = {}
        self.initialize()
    
    # Q値（各状態における各行動の価値）を初期化
    def initialize(self):
        for s in range(self.env.nS):
            self.Q[s] = [0] * self.env.nA
        self.env.reset()

In [6]:
mc = MonteCarlo(env)
mc.Q

{0: [0, 0, 0, 0],
 1: [0, 0, 0, 0],
 2: [0, 0, 0, 0],
 3: [0, 0, 0, 0],
 4: [0, 0, 0, 0],
 5: [0, 0, 0, 0],
 6: [0, 0, 0, 0],
 7: [0, 0, 0, 0],
 8: [0, 0, 0, 0],
 9: [0, 0, 0, 0],
 10: [0, 0, 0, 0],
 11: [0, 0, 0, 0],
 12: [0, 0, 0, 0],
 13: [0, 0, 0, 0],
 14: [0, 0, 0, 0],
 15: [0, 0, 0, 0]}

## 3.1モンテカルロ法の理論
Q(s,a)は、「状態sでaという行動をしたときに、ゲーム終了までに得られる報酬」で決める。

$$G_t = r_{t+1} + r_{t+2} + \cdots + r_T$$

今回はゲーム終了時にしか報酬をもらえないため、結局は$G_t = r_T$となる

ただ、未来でもらえる報酬は不確かな値なので、「割引」$\gamma$をかける。
$$G_t = r_{t+1} + \gamma r_{t+2} + \gamma^2 r_{t+3} + \cdots + \gamma^{T-t-1} r_T$$
これを、「割引現在価値」という。
$$Q(s,a)  Q(s,a) + \alpha (G_t - Q(s,a))$$

$\alpha$は、「学習率」で0から1を取る。Qの更新は1ゲーム終了したら行われる。

In [19]:
class MonteCarlo:
    def __init__(self, env):
        self.env = env
        self.Q = {}
        self.initialize()
    
    # Q値（各状態における各行動の価値）を初期化
    def initialize(self):
        for s in range(self.env.nS):
            self.Q[s] = [0] * self.env.nA
        self.env.reset()
    
    def learn(self, episode_count=10, gamma=0.9):
        for e in range(episode_count):
            done = False
            experience_log = []
            s = 0
            self.env.reset()
            
            # エピソードを実行
            while not done:
                # とりあえずランダムに行動を選択
                a = np.random.randint(self.env.nA)
                # 行動を実行して、次の状態:next_s、報酬:reward、終了したかどうか:doneを取得する
                next_s, reward, done, _ = self.env.step(a)
                #　「現在の状態」から「ある行動」をとったときの「価値」をログとして保存する
                experience_log.append({'state':s, 'action':a, 'reward':reward})
                s = next_s
            
            # エピソードで取得したログからQ値を更新
            for t, experience in enumerate(experience_log):
                G = 0
                i = 0
                for j in range(t, len(experience_log)):
                    G += (gamma**i) * experience_log[j]['reward']
                    i += 1
                s = experience['state']
                a = experience['action']
                self.Q[s][a] += alpha * (G)
        return experience_log

In [22]:
mc = MonteCarlo(env)
mc.learn()

[{'state': 0, 'action': 3, 'reward': 0.0},
 {'state': 0, 'action': 0, 'reward': 0.0},
 {'state': 0, 'action': 1, 'reward': 0.0},
 {'state': 4, 'action': 3, 'reward': 0.0},
 {'state': 0, 'action': 1, 'reward': 0.0},
 {'state': 4, 'action': 3, 'reward': 0.0},
 {'state': 0, 'action': 1, 'reward': 0.0},
 {'state': 4, 'action': 3, 'reward': 0.0},
 {'state': 0, 'action': 3, 'reward': 0.0},
 {'state': 0, 'action': 0, 'reward': 0.0},
 {'state': 0, 'action': 1, 'reward': 0.0},
 {'state': 4, 'action': 3, 'reward': 0.0},
 {'state': 0, 'action': 1, 'reward': 0.0},
 {'state': 4, 'action': 3, 'reward': 0.0},
 {'state': 0, 'action': 3, 'reward': 0.0},
 {'state': 0, 'action': 0, 'reward': 0.0},
 {'state': 0, 'action': 3, 'reward': 0.0},
 {'state': 0, 'action': 1, 'reward': 0.0},
 {'state': 4, 'action': 3, 'reward': 0.0},
 {'state': 0, 'action': 3, 'reward': 0.0},
 {'state': 0, 'action': 1, 'reward': 0.0},
 {'state': 4, 'action': 3, 'reward': 0.0},
 {'state': 0, 'action': 3, 'reward': 0.0},
 {'state': 

In [23]:
env.reset()
env.step(1)

(4, 0.0, False, {'prob': 1.0})

In [24]:
env.render()

  (Down)
SFFF
FHFH
FFFH
HFFG


In [25]:
env.step(2)

(5, 0.0, True, {'prob': 1.0})

In [26]:
env.render()

  (Right)
SFFF
FHFH
FFFH
HFFG


In [27]:
env.reset()
env.step(1)
env.step(1)
env.step(2)
env.step(1)
env.step(2)

(14, 0.0, False, {'prob': 1.0})

In [28]:
env.render()

  (Right)
SFFF
FHFH
FFFH
HFFG


In [29]:
env.step(2)

(15, 1.0, True, {'prob': 1.0})

In [30]:
env.render()

  (Right)
SFFF
FHFH
FFFH
HFFG


In [31]:
env.step(3)

(15, 0, True, {'prob': 1.0})

In [32]:
env.step(0)

(15, 0, True, {'prob': 1.0})